In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import math

In [2]:
# model parameters

Ka = 1e-5
c_bufs = {'Tris':1.1e-3,'TrisH': 8.9e-3}
zs = {'K': 1,'Cl': -1,'H': 1,'OH': -1}
Ps = {'K': 3e-6,'Cl': (3/6.5)*1e-6,'H': 0,'OH': 0} #m^s-1
c = 1e-2 #Fm^-2

F = 96485 #Cmol^-1
R = 8.31 #JK^-1mol^-1
r_v = 12.5e-6 #m
T = 298 #K
A = 4*np.pi*r_v**2
vol = 4*np.pi*r_v**3/3
c_outs = {'K':1,'Cl':1,'H':0,'OH':0} #Molm^-3


B = np.log(10)*20*Ka*1e-4/(2e-4)**2 #buffer capacity for my buffer. TODO: measure this!
C = c*A
eta = (zs['H']**2)*(F**2)*vol*B/(R*T*np.log(10)*C)
# eta = 1e6

# pHo = -1*math.log(c_outs['H'])/(math.log(10))
    
fluxes = {key:[] for key in Ps.keys()}
dCs = {key:[] for key in Ps.keys()}
threshold = 0.1

c_ins = {key:[] for key in Ps.keys()}

V = []

dts = np.logspace(-4,-2,3) #s
# dts = [1e-6]

# Old model: seemed to be unstable

def initialise():
    
    V = 0
    
    c = {}
    c['K'] = 0
    c['Cl'] = 0
    c['H'] = 1e-4
    c['OH'] = 1e-4
    return V,c
    

def dCdt(c_in,c_out,P,V, z):
    
    mu = z*F*V/(R*T)
#     print('mu = ', mu)
#     print(abs(mu))
#     print('mu: ',abs(mu))
    if abs(mu) < threshold:
        
        return (P/(3*r_v))*(c_out - c_in)
    
#     print('mu = ', mu)
    
    prefactor = mu*P/(3*r_v)
    
#     print('exp: ',np.exp(-mu))
    return prefactor*(c_out - c_in*np.exp(mu))/(np.exp(mu)-1)


def dV(dCs,vol,C,zs):
    
    # args:
    # dCs: dictionary of ion molar fluxes
    # vol: volume of GUV
    # eta: dimensionless constant. the voltage change due to one unit of pH change
    # C: capacitance of GUV membrane
    
    prefactor = 1/C
    
    total_Q_flux =0
    for key in dCs.keys():

            
        total_Q_flux += zs[key]*F*vol*dCs[key]
        
#     print(total_Q_flux)
#     print('total charge flux: ',total_Q_flux)
    return prefactor*total_Q_flux


def adjustcH(Ka,c_bufs):

    
    return Ka*c_bufs['TrisH']/c_bufs['Tris']


"""
def calcV(Q,C,A):
#     print(Q)
    return Q/(C*A)


def Q_in(c_ins,vol):
    
    Q = 0 
    for key in c_ins.keys():
#         print(c_ins[key])
        Q += vol*F*zs[key]*c_ins[key][-1]
        
#     print('Q = ',Q)
    return Q
"""    

# def cH(V,pHo):
    

    
#     neg_pH = F*V[-1]/(R*T*math.log(10))-pHo
# #     print('pH inside GUV: ',-neg_pH)
#     return 10**neg_pH

def step_forward(c_latest,V):
    
    # calculate fluxes and increment the internal concentrations
    
    next_flux = {}
    next_dCs = {}
    for key in c_latest.keys():

            
        dC = dt*dCdt(c_latest[key],c_outs[key],Ps[key],V,zs[key])
        next_flux[key] = dC/dt
        next_dCs[key] = dC
        
        
#         c_ins[key].append(c_ins[key][-1]+dC)
#         c_ins['H'][-1] = adjustcH(Ka,c_bufs)
        
        
        c_latest[key] = c_latest[key]+ dC
        
    # adjust the H+ concentration due to the buffer
    
#     dcH = c_ins['H'][-2] - c_ins['H'][-1]
    
#     c_ins['H'][-1] += adjustcH(c_ins['H'][-1],Ka,c_bufs['-'][-1],dcH)
    
    
#     Q = Q_in(c_ins,vol)
#     print(Q)

    
#     V.append(V[-1]+dV(dCs,vol,eta,C,zs))
    V  = V + dV(next_dCs,vol,C,zs)
#     print('V: ',V[-1])
    return V,c_latest,next_flux,next_dCs



In [9]:
Ps['K']/Ps['Cl']

6.5

In [3]:
#define times

ts = np.arange(0,20000000)

# initialise


initialise()

UPDATE_INTERVAL = 10

V_mem = np.zeros((ts.shape[0]//UPDATE_INTERVAL,))
for key in c_ins.keys():
    c_ins[key] = np.zeros_like(V_mem)
    fluxes[key] = np.zeros_like(V_mem)
    dCs[key] = np.zeros_like(V_mem)
    
# start time loop
counter = 0

t1 = 60000000
t2 = 70000000
for t in ts:
    dt = 2*dts[0]
    if t >t1:
        dt = dts[1]
    if t > t2:
        dt = dts[2]
    
    index = int(((t- t%UPDATE_INTERVAL)/UPDATE_INTERVAL + t%UPDATE_INTERVAL))
    
    if not t:
        V,c_latest = initialise()
        for key in c_ins.keys():
            c_ins[key][t] = c_latest[key]
            
        V_mem[t] = V

        
    V, nextc_ins,next_flux,next_dCs = step_forward(c_latest,V)
    V_mem[index] = V
    
    if not t % UPDATE_INTERVAL and t > 0:
        counter += 1
        
        V_mem[int((t-UPDATE_INTERVAL)/UPDATE_INTERVAL)+1] = V
#         print(V)
        for key in c_ins.keys():
#             lastC = c_ins[key][int(t/5)]
            c_ins[key][int((t-UPDATE_INTERVAL)/UPDATE_INTERVAL)+1] = nextc_ins[key]
            
#             lastflux = fluxes[key][int(t/5)]
            fluxes[key][int((t-UPDATE_INTERVAL)/UPDATE_INTERVAL)+1] = next_flux[key]
            dCs[key][int((t-UPDATE_INTERVAL)/UPDATE_INTERVAL)+1] = next_dCs[key]
    if not t% 100000:
        print(t, ' steps completed')
    
    c_latest = nextc_ins
    

0  steps completed
100000  steps completed
200000  steps completed
300000  steps completed
400000  steps completed
500000  steps completed
600000  steps completed
700000  steps completed
800000  steps completed
900000  steps completed
1000000  steps completed
1100000  steps completed
1200000  steps completed
1300000  steps completed
1400000  steps completed
1500000  steps completed
1600000  steps completed
1700000  steps completed
1800000  steps completed
1900000  steps completed
2000000  steps completed
2100000  steps completed
2200000  steps completed
2300000  steps completed
2400000  steps completed
2500000  steps completed
2600000  steps completed
2700000  steps completed
2800000  steps completed
2900000  steps completed
3000000  steps completed
3100000  steps completed
3200000  steps completed
3300000  steps completed
3400000  steps completed
3500000  steps completed
3600000  steps completed
3700000  steps completed
3800000  steps completed
3900000  steps completed
4000000  steps 

IndexError: index 2000000 is out of bounds for axis 0 with size 2000000

In [ ]:
len(ts), len(V_mem)

In [4]:
indices = np.arange(0,len(ts),UPDATE_INTERVAL)

In [ ]:
indices

In [7]:
%matplotlib qt
fig,axs = plt.subplots(1,5)


t = np.zeros_like(indices).astype(float)
t[indices <=t1] = 2*dts[0]*indices[indices <=t1]
t[(indices > t1) * (indices <=t2)] = t[indices == t1] +dts[1]*(indices[(indices > t1) * (indices <=t2)]-indices[indices == t1])
t[indices >t2] = t[indices == t2] + dts[2]*(indices[indices >t2]-indices[indices==t2])

for i,key in enumerate(c_ins.keys()):

        #
        #
    axs[i].plot(t,c_ins[key])
    
    axs[i].xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    
    axs[i].set_title(key,fontsize = 18)
    axs[i].set_xlabel('T [mins]')
#     axs[i].set_xticks([0,1000,2000])
#     axs[i].set_xticklabels([0,1000//60,2000//60],fontsize = 15)
    axs[i].set_xlim([-1,51])

    
# axs[0].set_ylim([0,1])
# axs[1].set_ylim([0,1])
axs[4].plot(t,V_mem)

axs[4].set_title('V_m')
# axs[4].set_xlim([-1,51])
# axs[3].set_xticks([0,1000,2000])
# axs[3].set_xticklabels([0,1000//60,2000//60],fontsize = 15)
# axs[3].set_xlim([-50,2050])
# axs[3].set_ylim([0,1])

Text(0.5, 1.0, 'V_m')

In [5]:
def set_plot_params(ax,xticks,yticks,
                    xlabel,ylabel,x_sciformat = 1,y_sciformat = 1):
    plt.tick_params(direction = 'in',top = True,right = True,left = True,bottom = True, length = 6)


    if x_sciformat != 1 or y_sciformat != 1:
        xticklabels = np.array(xticks)/x_sciformat
        yticklabels = np.array(yticks)/y_sciformat
        
    else:
        xticklabels = xticks
        yticklabels = yticks
        
    ax.set_xticks(xticks)
    ax.set_xticklabels(xticklabels,fontsize = 16)

    ax.set_yticks(yticks)
    ax.set_yticklabels(yticklabels,fontsize = 16)

    ax.set_xlabel(xlabel,fontsize = 18)
    ax.set_ylabel(ylabel,fontsize = 18)


In [7]:
%matplotlib qt
fig,axs = plt.subplots(1,1)


t = np.zeros_like(indices).astype(float)
t[indices <=t1] = 2*dts[0]*indices[indices <=t1]
t[(indices > t1) * (indices <=t2)] = t[indices == t1] +dts[1]*(indices[(indices > t1) * (indices <=t2)]-indices[indices == t1])
t[indices >t2] = t[indices == t2] + dts[2]*(indices[indices >t2]-indices[indices==t2])

for i,key in enumerate(['K','Cl']):

    
        
    axs.plot(t,c_ins[key],c = 'C'+str(i+3))
    
#     axs[0].xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    
#     axs.set_title(key,fontsize = 18)
    axs.set_xlabel('T [mins]')
#     axs[i].set_xticks([0,1000,2000])
#     axs[i].set_xticklabels([0,1000//60,2000//60],fontsize = 15)
    axs.set_xlim([-0.01,0.11])

    xticks = [0,0.05,0.1]
    yticks = [0,0.005,0.01]
    
    xlabel = 'T [mins]'
    ylabel = '$C_i$ [mM]'
    
    x_sciformat = 1/60
    set_plot_params(axs,xticks,yticks,xlabel,ylabel,x_sciformat=x_sciformat)
    
    axs.set_xlim([-0.01,0.11])
    axs.set_ylim([-0.001,0.011])
# axs[0].set_ylim([0,1])
# axs[1].set_ylim([0,1])
# axs[1].plot(t,V_mem)

# axs[1].set_title('V_m')
# axs[1].set_xlim([-1,51])
# axs[3].set_xticks([0,1000,2000])
# axs[3].set_xticklabels([0,1000//60,2000//60],fontsize = 15)
# axs[3].set_xlim([-50,2050])
# axs[3].set_ylim([0,1])

In [10]:
fig, ax = plt.subplots(1,1)

ax.plot(t,V_mem,c = 'C6')

xlabel = 'T [mins]'
ylabel = '$V_{mem}$ [mV]'

xticks = [0,25,50]
yticks = [0,0.03,0.06]
y_sciformat = 1e-3

ax.set_xlim([-1,51])
ax.set_ylim([-0.001,0.061])
set_plot_params(ax,xticks,yticks,xlabel,ylabel,y_sciformat = y_sciformat)

In [ ]:
print(indices[indices <=8e7])
print(indices[(indices > 8e7) * (indices <=9e7)]-indices[indices==8e7])

In [29]:
%matplotlib qt

In [ ]:
%matplotlib qt
fig, ax = plt.subplots(1,5)

for i,key in enumerate(c_ins.keys()):
    ax[i].plot(c_ins[key])

ax[i+1].plot(V_mem)

In [13]:
# %matplotlib qt
fig,axs = plt.subplots(1,1)

for key in ['K','Cl']:
    
    axs.scatter(1-np.array(c_ins[key]),fluxes[key])
    
    axs.set_title(key)

In [ ]:
# equilibrium ion number in GUV
N = vol #moles
N = N*(6e23) #ions

In [ ]:
init_flux = fluxes['K'][0]/100 #molsm^-2s^-1
init_transport_rate = init_flux*A #molss^-1
init_transport_rate = (6e23)*init_transport_rate


In [ ]:
fig, ax = plt.subplots(1,1)
ax.plot(t[t<t1],dCs['K'][t<t1])

In [ ]:
print(len(dCs['K']),len(dCs['K'][t<t1]))

In [ ]:
N/init_transport_rate

In [ ]:
# define 4th order Runge - Kutta method for updating concentrations

def dCrk(dt,c_latest,V):
    K0 = {}
    K1 = {}
    K2 = {}
    K3 = {}
    
    for key in c_latest.keys():
        K0[key] = dt*dCdt(c__latest[key],c_outs[key],Ps[key],V, zs[key])
    
    V1 = V + dV(K0[key]/2,vol,C,zs)
    
    for key in c_latest.keys():
        
        K1[key] = dt*dCdt(c__latest[key]+K0[key]/2,c_outs[key],Ps[key],V1, zs[key])
    
    V2 = V + dV(K1[key]/2,vol,C,zs)
    
    for key in c_latest.keys():
        
        K2[key] = dt*dCdt(c__latest[key]+K1[key]/2,c_outs[key],Ps[key],V2, zs[key])
    
    V3 = V+ dV(K2[key],vol,C,zs)
    
    for key in c_latest.keys():
        
        K2[key] = dt*dCdt(c__latest[key]+K1[key]/2,c_outs[key],Ps[key],V2, zs[key])
        
    for key in c_latest.keys():
        c_latest[key] = c_latest[key] + (1/6)*(K0[key]+2*K1[key]+2*K2[key]+K3[key])
        
    return c_latest